# BinanceBot - Kaggle Training Notebook

**Strategy:** UniversalMACD  
**Epochs:** 100 per timeframe  
**Timeframes:** 4h, 12h, 1d  
**Total Runtime:** ~21-22 hours

## Workflow:
1. Install BinanceBot from uploaded dataset
2. Download historical data
3. Run Hyperopt (optimize parameters)
4. Run Backtest (test on unseen data)
5. Save and download results

## Prerequisites:
- Upload `kaggle_package` folder as Kaggle Dataset named `binancebot-package`
- Add dataset to this notebook

In [ ]:
# CELL 1: Install BinanceBot from GitHub (Test Repository)
import subprocess
import sys

# Change this URL when ready to use main repo
GITHUB_REPO = "https://github.com/NDuc1602/binancebot.git"

print(f"Installing BinanceBot from: {GITHUB_REPO}")
print("This will take 3-5 minutes...")

# Install directly from GitHub repo
result = subprocess.run([
    sys.executable, "-m", "pip", "install", 
    "-q",
    f"git+{GITHUB_REPO}"
], capture_output=True, text=True)

if result.returncode == 0:
    print("✓ Installation successful!")
else:
    print("⚠ Installation completed with warnings (normal on Kaggle)")
    if "Successfully installed" in result.stdout or "Successfully installed" in result.stderr:
        print("✓ BinanceBot was installed despite warnings")

# Verify installation
print("\nVerifying BinanceBot...")
result = subprocess.run(['binancebot', '--version'], capture_output=True, text=True)
if result.returncode == 0:
    print(f"✓ BinanceBot {result.stdout.strip()}")
    print("✓ Ready to start!")
else:
    print("✗ Installation verification failed")
    print("Trying alternative check...")
    result2 = subprocess.run([sys.executable, "-m", "binancebot", "--version"], capture_output=True, text=True)
    if result2.returncode == 0:
        print(f"✓ BinanceBot {result2.stdout.strip()}")
        print("✓ Ready (use 'python -m binancebot' instead of 'binancebot')")
    else:
        raise RuntimeError("BinanceBot installation failed")

In [ ]:
# CELL 2: Import Libraries
import subprocess
import json
import shutil
from pathlib import Path
from datetime import datetime

print("✓ Libraries imported")

In [ ]:
# CELL 3: Configuration
CONFIG = {
    'strategy': 'UniversalMACD',
    'timeframes': ['4h', '12h', '1d'],
    'pairs': ['BTC/USDT', 'ETH/USDT', 'BNB/USDT'],
    'epochs': 100,
    'loss_function': 'SharpeHyperOptLoss',
    'hyperopt_timerange': '20200101-20231231',  # Train: 2020-2023 (4 years)
    'backtest_timerange': '20240101-20251031',  # Test: 2024-2025 (22 months)
}

# Paths
WORK_DIR = Path('/kaggle/working')
USER_DATA = WORK_DIR / 'user_data'
CONFIG_PATH = WORK_DIR / 'config.json'

print("Configuration:")
print(f"  Strategy: {CONFIG['strategy']}")
print(f"  Timeframes: {CONFIG['timeframes']}")
print(f"  Epochs: {CONFIG['epochs']}")
print(f"  Pairs: {CONFIG['pairs']}")
print(f"  Total runs: {len(CONFIG['timeframes'])}")
print(f"  Estimated time: ~7h per timeframe = ~21h total")

In [ ]:
# CELL 4: Create Config File
config_data = {
    "max_open_trades": 3,
    "stake_currency": "USDT",
    "stake_amount": 30,
    "dry_run": True,
    "timeframe": "4h",
    "exchange": {
        "name": "binance",
        "pair_whitelist": CONFIG['pairs'],
        "pair_blacklist": []
    },
    "pairlists": [{"method": "StaticPairList"}],
    "entry_pricing": {
        "price_side": "same",
        "use_order_book": True,
        "order_book_top": 1,
        "check_depth_of_market": {
            "enabled": False,
            "bids_to_ask_delta": 1
        }
    },
    "exit_pricing": {
        "price_side": "same",
        "use_order_book": True,
        "order_book_top": 1
    }
}

CONFIG_PATH.write_text(json.dumps(config_data, indent=2))
print(f"✓ Config created: {CONFIG_PATH}")

In [ ]:
# CELL 5: Create Strategy File
strategy_code = '''
import numpy as np
import talib.abstract as ta
from pandas import DataFrame
from binancebot.strategy import IStrategy, IntParameter

class UniversalMACD(IStrategy):
    """
    Universal MACD Strategy with hyperoptable parameters
    """
    
    minimal_roi = {
        "0": 0.10,
        "30": 0.05,
        "60": 0.02,
        "120": 0.01
    }
    
    stoploss = -0.10
    trailing_stop = False
    
    # Hyperoptable parameters
    buy_macd_fast = IntParameter(8, 16, default=12, space='buy')
    buy_macd_slow = IntParameter(20, 30, default=26, space='buy')
    
    sell_macd_fast = IntParameter(8, 16, default=12, space='sell')
    sell_macd_slow = IntParameter(20, 30, default=26, space='sell')
    
    timeframe = '4h'
    
    def populate_indicators(self, dataframe: DataFrame, metadata: dict) -> DataFrame:
        # MACD with buy parameters
        macd_buy = ta.MACD(dataframe, 
                          fastperiod=self.buy_macd_fast.value,
                          slowperiod=self.buy_macd_slow.value,
                          signalperiod=9)
        dataframe['macd_buy'] = macd_buy['macd']
        dataframe['macdsignal_buy'] = macd_buy['macdsignal']
        
        # MACD with sell parameters  
        macd_sell = ta.MACD(dataframe,
                           fastperiod=self.sell_macd_fast.value,
                           slowperiod=self.sell_macd_slow.value,
                           signalperiod=9)
        dataframe['macd_sell'] = macd_sell['macd']
        dataframe['macdsignal_sell'] = macd_sell['macdsignal']
        
        return dataframe
    
    def populate_entry_trend(self, dataframe: DataFrame, metadata: dict) -> DataFrame:
        dataframe.loc[
            (dataframe['macd_buy'] > dataframe['macdsignal_buy']),
            'enter_long'] = 1
        return dataframe
    
    def populate_exit_trend(self, dataframe: DataFrame, metadata: dict) -> DataFrame:
        dataframe.loc[
            (dataframe['macd_sell'] < dataframe['macdsignal_sell']),
            'exit_long'] = 1
        return dataframe
'''

# Create strategy folder
strategy_dir = USER_DATA / 'strategies'
strategy_dir.mkdir(parents=True, exist_ok=True)
strategy_file = strategy_dir / 'UniversalMACD.py'
strategy_file.write_text(strategy_code)

print(f"✓ Strategy created: {strategy_file}")

In [ ]:
# CELL 6: Download Data
def download_data(timeframe):
    """Download historical data"""
    print(f"\n📥 Downloading {timeframe} data...")
    
    cmd = [
        sys.executable, '-m', 'binancebot', 'download-data',
        '--pairs', *CONFIG['pairs'],
        '--timeframe', timeframe,
        '--timerange', CONFIG['hyperopt_timerange'],
        '--config', str(CONFIG_PATH),
        '--userdir', str(USER_DATA)
    ]
    
    result = subprocess.run(cmd, capture_output=True, text=True)
    if result.returncode == 0:
        print(f"✓ {timeframe} downloaded")
    else:
        print(f"✗ Error downloading {timeframe}")
        print(result.stderr[-500:])

# Download all timeframes
print("Downloading data for all timeframes...")
import sys
for tf in CONFIG['timeframes']:
    download_data(tf)

print("\n✓ Data download complete!")

In [ ]:
# CELL 7: Run Hyperopt & Backtest
import sys

def run_hyperopt(timeframe):
    """Run hyperopt for one timeframe"""
    print(f"\n{'='*70}")
    print(f"🔧 HYPEROPT: {CONFIG['strategy']} @ {timeframe}")
    print(f"   Epochs: {CONFIG['epochs']} | Loss: {CONFIG['loss_function']}")
    print(f"{'='*70}")
    
    cmd = [
        sys.executable, '-m', 'binancebot', 'hyperopt',
        '--strategy', CONFIG['strategy'],
        '--timeframe', timeframe,
        '--timerange', CONFIG['hyperopt_timerange'],
        '--epochs', str(CONFIG['epochs']),
        '--hyperopt-loss', CONFIG['loss_function'],
        '--config', str(CONFIG_PATH),
        '--userdir', str(USER_DATA),
        '--spaces', 'buy', 'sell',
    ]
    
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1
    )
    
    epoch = 0
    all_output = []
    for line in process.stdout:
        all_output.append(line)
        if 'Best result:' in line:
            print(line.rstrip())
        elif 'epoch' in line.lower() and '/' in line:
            epoch += 1
            if epoch % 10 == 0:
                print(f"   Progress: {epoch}/{CONFIG['epochs']} epochs...")
        elif 'Objective:' in line:
            print(line.rstrip())
        # Print errors immediately
        elif any(keyword in line.lower() for keyword in ['error', 'exception', 'traceback', 'failed']):
            print(line.rstrip())
    
    process.wait()
    
    if process.returncode == 0:
        print(f"\n✅ Hyperopt COMPLETE: {timeframe}")
        return True
    else:
        print(f"\n❌ Hyperopt FAILED: {timeframe}")
        print("\n🔍 Full error output:")
        print(''.join(all_output[-50:]))  # Print last 50 lines
        return False

def run_backtest(timeframe):
    """Run backtest for one timeframe"""
    print(f"\n{'='*70}")
    print(f"📊 BACKTEST: {CONFIG['strategy']} @ {timeframe}")
    print(f"   Timerange: {CONFIG['backtest_timerange']}")
    print(f"{'='*70}")
    
    cmd = [
        sys.executable, '-m', 'binancebot', 'backtesting',
        '--strategy', CONFIG['strategy'],
        '--timeframe', timeframe,
        '--timerange', CONFIG['backtest_timerange'],
        '--config', str(CONFIG_PATH),
        '--userdir', str(USER_DATA)
    ]
    
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True,
        bufsize=1
    )
    
    output_lines = []
    for line in process.stdout:
        output_lines.append(line)
        if any(keyword in line for keyword in [
            'BACKTESTING REPORT', 'Total/Daily', 'Win', 'Loss', 'Avg profit',
            'Total profit', 'Trades', 'Drawdown'
        ]):
            print(line.rstrip())
    
    process.wait()
    
    if process.returncode == 0:
        print(f"\n✅ Backtest COMPLETE: {timeframe}")
        metrics = parse_backtest_output(''.join(output_lines))
        return True, metrics
    else:
        print(f"\n❌ Backtest FAILED: {timeframe}")
        return False, {}

def parse_backtest_output(output):
    """Parse backtest output to extract metrics"""
    metrics = {}
    for line in output.split('\n'):
        if 'Total profit' in line or 'Absolute profit' in line:
            parts = line.split('|')
            if len(parts) >= 2:
                metrics['total_profit'] = parts[1].strip()
        elif 'Total trades' in line:
            parts = line.split('|')
            if len(parts) >= 2:
                metrics['trades'] = parts[1].strip()
        elif 'Max Drawdown' in line:
            parts = line.split('|')
            if len(parts) >= 2:
                metrics['max_drawdown'] = parts[1].strip()
    return metrics

# Run pipeline for all timeframes
results = {}
all_metrics = []

for idx, tf in enumerate(CONFIG['timeframes'], 1):
    print(f"\n\n{'#'*70}")
    print(f"# COMBINATION {idx}/{len(CONFIG['timeframes'])}: {tf}")
    print(f"{'#'*70}")
    
    # Hyperopt
    hyperopt_success = run_hyperopt(tf)
    
    # Backtest
    backtest_success = False
    metrics = {}
    
    if hyperopt_success:
        backtest_success, metrics = run_backtest(tf)
    
    # Store results
    results[tf] = {
        'hyperopt': 'SUCCESS' if hyperopt_success else 'FAILED',
        'backtest': 'SUCCESS' if backtest_success else 'FAILED',
        'metrics': metrics
    }
    
    if backtest_success:
        all_metrics.append({'timeframe': tf, **metrics})
    
    # Progress summary
    print(f"\n📊 PROGRESS SUMMARY:")
    for timeframe, data in results.items():
        h_icon = '✅' if data['hyperopt'] == 'SUCCESS' else '❌'
        b_icon = '✅' if data['backtest'] == 'SUCCESS' else '❌'
        print(f"   {timeframe}: Hyperopt {h_icon} | Backtest {b_icon}")
        if data['metrics']:
            print(f"      Profit: {data['metrics'].get('total_profit', 'N/A')}")
            print(f"      Trades: {data['metrics'].get('trades', 'N/A')}")

In [ ]:
# CELL 8: Save Results
output_dir = WORK_DIR / 'training_results'
output_dir.mkdir(exist_ok=True)

# Copy hyperopt results
hyperopt_dir = USER_DATA / 'hyperopt_results'
if hyperopt_dir.exists():
    for file in hyperopt_dir.glob('*.fthypt'):
        shutil.copy(file, output_dir)
        print(f"✓ Saved: {file.name}")

# Copy optimized parameters
strategy_json = USER_DATA / 'strategies' / f'{CONFIG["strategy"]}.json'
if strategy_json.exists():
    shutil.copy(strategy_json, output_dir)
    print(f"✓ Saved: {strategy_json.name}")

# Copy backtest results
backtest_dir = USER_DATA / 'backtest_results'
if backtest_dir.exists():
    for file in backtest_dir.glob('*.json'):
        shutil.copy(file, output_dir)
        print(f"✓ Saved: {file.name}")

# Create summary file
summary_file = output_dir / 'complete_summary.txt'
with open(summary_file, 'w', encoding='utf-8') as f:
    f.write(f"Complete Training & Backtest Summary\n")
    f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"{'='*70}\n\n")
    f.write(f"Strategy: {CONFIG['strategy']}\n")
    f.write(f"Epochs: {CONFIG['epochs']}\n")
    f.write(f"Timeframes: {', '.join(CONFIG['timeframes'])}\n")
    f.write(f"Train Period: {CONFIG['hyperopt_timerange']}\n")
    f.write(f"Test Period: {CONFIG['backtest_timerange']}\n\n")
    
    f.write("="*70 + "\n")
    f.write("RESULTS BY TIMEFRAME\n")
    f.write("="*70 + "\n\n")
    
    for tf, data in results.items():
        f.write(f"\nTimeframe: {tf}\n")
        f.write(f"  Hyperopt: {data['hyperopt']}\n")
        f.write(f"  Backtest: {data['backtest']}\n")
        
        if data['metrics']:
            f.write(f"  Metrics:\n")
            for key, value in data['metrics'].items():
                f.write(f"    {key}: {value}\n")
        f.write("\n")

print(f"\n✓ Results saved to: {output_dir}")

# Export metrics to CSV
if all_metrics:
    import csv
    csv_file = output_dir / 'backtest_metrics.csv'
    
    all_keys = set()
    for m in all_metrics:
        all_keys.update(m.keys())
    
    with open(csv_file, 'w', newline='', encoding='utf-8') as f:
        writer = csv.DictWriter(f, fieldnames=sorted(all_keys))
        writer.writeheader()
        writer.writerows(all_metrics)
    
    print(f"✓ Metrics CSV saved: {csv_file}")

print(f"\n📥 Download 'training_results' folder from Output tab!")

In [ ]:
# CELL 9: Display Summary
print("\n" + "="*70)
print("🎉 TRAINING & BACKTEST COMPLETE!")
print("="*70)

print(f"\nStrategy: {CONFIG['strategy']}")
print(f"Epochs per timeframe: {CONFIG['epochs']}")
print(f"Total timeframes: {len(CONFIG['timeframes'])}")

print(f"\n📊 RESULTS SUMMARY:")
print("-"*70)

for tf, data in results.items():
    h_status = '✅' if data['hyperopt'] == 'SUCCESS' else '❌'
    b_status = '✅' if data['backtest'] == 'SUCCESS' else '❌'
    
    print(f"\n{tf}:")
    print(f"  Hyperopt: {h_status}")
    print(f"  Backtest: {b_status}")
    
    if data['metrics']:
        print(f"  Metrics:")
        for key, value in data['metrics'].items():
            print(f"    - {key}: {value}")

print("\n" + "="*70)
print(f"📂 Output folder: {output_dir}")
print("="*70)
print("\n📥 Download from Kaggle Output tab:")
print("  - *.fthypt files (hyperopt history)")
print("  - *.json files (backtest results + optimized params)")
print("  - complete_summary.txt (full summary)")
print("  - backtest_metrics.csv (metrics table)")
print("\n💡 Use these files locally:")
print("  1. Copy *.fthypt to user_data/hyperopt_results/")
print("  2. Copy UniversalMACD.json to user_data/strategies/")
print("  3. Run backtest locally with optimized params")